# Tool Asta Fantacalcio

Per prima cosa occorre scaricare i due dataset che verranno utilizzati:
- quotazioni
- votazioni

In [1]:
'''
quotazioni:
https://www.fantacalcio.it/quotazioni-fantacalcio

votazioni:
https://www.fantacalcio.it/statistiche-serie-a/2021-22/fantacalcio/medie
'''

'\nquotazioni:\nhttps://www.fantacalcio.it/quotazioni-fantacalcio\n\nvotazioni:\nhttps://www.fantacalcio.it/statistiche-serie-a/2021-22/fantacalcio/medie\n'

In [2]:
import pandas as pd
import os

In [3]:
quotazioni_df = pd.read_excel("data/input/Quotazioni_Fantacalcio.xlsx", header = 1, engine='openpyxl')
quotazioni_df.head()

,Id,R,Nome,Squadra,Qt. A,Qt. I,Diff.
0,785,A,IMMOBILE,Lazio,44,38,6
1,2841,A,VLAHOVIC,Juventus,41,34,7
2,531,A,BERARDI,Sassuolo,40,31,9
3,608,A,ZAPATA D.,Atalanta,34,33,1
4,1958,A,CAPRARI,Verona,34,10,24


In [4]:
quotazioni_df.columns

Index(['Id', 'R', 'Nome', 'Squadra', 'Qt. A', 'Qt. I', 'Diff.'], dtype='object')

In [5]:
def prune(dataset):
    temp = pd.DataFrame()
    temp['Nome'] = dataset['Nome']
    temp['Pg'] = dataset['Pg']
    temp['Mf'] = dataset['Mf']
    return temp

Rimuovo colonne non utilizzate

In [6]:
quotazioni_df = quotazioni_df.drop(['Id', 'Diff.'], axis=1)

In [7]:
stat_df1 = pd.read_excel("data/input/Statistiche_Fantacalcio_2021-22.xlsx", header = 1, engine='openpyxl')
stat_df2 = pd.read_excel("data/input/Statistiche_Fantacalcio_2020-21.xlsx", header = 1, engine='openpyxl')
stat_df3 = pd.read_excel("data/input/Statistiche_Fantacalcio_2019-20.xlsx", header = 1, engine='openpyxl')

In [8]:
stat_df1.head(2)

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
0,3,P,RADUNOVIC,Cagliari,3,6.33,5.67,0,5,1,0,0,0,0,0,0,0
1,4,P,SPORTIELLO,Atalanta,4,6.25,5.50,0,3,0,0,0,0,0,0,0,0


In [9]:
stat_df2.head(2)

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
0,3,P,RADUNOVIC,Atalanta,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
1,4,P,SPORTIELLO,Atalanta,15,6.23,5.03,0,21,1,0,0,0,0,0,0,0


In [10]:
stat_df3.head(2)

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
0,3,P,RADUNOVIC,Verona,3,5.83,3.00,0,8,0,0,0,0,0,1,0,0
1,4,P,SPORTIELLO,Atalanta,6,6.17,5.33,0,5,0,0,0,0,0,0,0,0


In [11]:
calciatori_oggi = list(stat_df1['Nome'] + '/' + stat_df1['Squadra'])
calciatori_old = list(stat_df2['Nome'] + '/' + stat_df2['Squadra'])

In [12]:
calciatori_old_dict = dict(zip(stat_df2['Nome'].to_list(), stat_df2['Squadra'].to_list()))

In [13]:
giveatry = [i.split('/')[0] for i in calciatori_oggi if i not in calciatori_old]

In [14]:
list(stat_df1.columns.values)

['Id',
 'R',
 'Nome',
 'Squadra',
 'Pg',
 'Mv',
 'Mf',
 'Gf',
 'Gs',
 'Rp',
 'Rc',
 'R+',
 'R-',
 'Ass',
 'Amm',
 'Esp',
 'Au']

In [15]:
stat_df1 = prune(stat_df1)
stat_df2 = prune(stat_df2)
stat_df3 = prune(stat_df3)

In [16]:
df_q = pd.DataFrame(quotazioni_df)
df_s1 = pd.DataFrame(stat_df1)
df_s2 = pd.DataFrame(stat_df2)
df_s3 = pd.DataFrame(stat_df3)

In [17]:
print(df_q.columns)
print(df_s1.columns)
print(df_s2.columns)
print(df_s3.columns)

Index(['R', 'Nome', 'Squadra', 'Qt. A', 'Qt. I'], dtype='object')
Index(['Nome', 'Pg', 'Mf'], dtype='object')
Index(['Nome', 'Pg', 'Mf'], dtype='object')
Index(['Nome', 'Pg', 'Mf'], dtype='object')


In [18]:
df = df_q.merge(df_s1, on='Nome').merge(df_s2, on='Nome').merge(df_s3, on='Nome')

In [19]:
df.head()

,R,Nome,Squadra,Qt. A,Qt. I,Pg_x,Mf_x,Pg_y,Mf_y,Pg,Mf
0,A,IMMOBILE,Lazio,44,38,23,8.98,35,7.70,37,9.49
1,A,VLAHOVIC,Juventus,41,34,27,8.56,37,8.01,27,6.46
2,A,BERARDI,Sassuolo,40,31,26,8.38,30,8.37,31,7.85
3,A,ZAPATA D.,Atalanta,34,33,17,8.38,37,7.88,28,8.66
4,A,CAPRARI,Verona,34,10,25,8.08,30,6.52,26,6.42


In [20]:
media_giocatori = [];
for index, row in df.iterrows():
    if row.Pg_x > 0 or row.Pg_y > 0:
        media_w_pg = (row.Pg_x/38 * row.Mf_x)*0.20 + (row.Pg_y/38 * row.Mf_y)*0.80
        media_giocatori.append(media_w_pg)
    else:
        media_giocatori.append(0)

df['mediaGiocatori'] = media_giocatori

In [21]:
media = []
for index, row in df.iterrows():
    if row.mediaGiocatori > 0:
        media.append(row.mediaGiocatori / row["Qt. I"])
    else:
        media.append(0)

df['media'] = media

In [22]:
giocateMax = 0
for index, row in df.iterrows():
    if row.Pg > giocateMax:
        giocateMax = row.Pg

print("In questa stagione sono state disputate", giocateMax ,"partite")

In questa stagione sono state disputate 38 partite


In [23]:
if giocateMax == 0:
    giocateMax = 1 # se campionato non iniziato evito divisione per zero

In [24]:
media_giocatori = [];
for index, row in df.iterrows():
    if row.Pg > 0:
        media_w_pg = (row.Pg_x/38 * row.Mf)*0.20 + (row.Pg_y/38 * row.Mf)*0.40 + (row.Pg/giocateMax * row.Mf)*0.40
        media_giocatori.append(media_w_pg)
    else:
        media_giocatori.append(0)

df['mediaGiocatori_today'] = media_giocatori

In [25]:
media = []
for index, row in df.iterrows():
    if row.mediaGiocatori_today > 0:
        media.append(row.mediaGiocatori / row["Qt. I"])
    else:
        media.append(0)

df['media_today'] = media

In [26]:
prob = []
for index, row in df.iterrows():
    if int(row.Pg_y) <= int(row.Pg_x):
        if round(int(row.Pg_y)/38*100-int(row.Pg_x+1)/int(row.Pg_y+1)) < 0 :
            prob.append(round(-1/2*(int(row.Pg_y)/38*100-(int(row.Pg_x+1)/int(row.Pg_y+1)))))
        else:
            prob.append(round(int(row.Pg_y)/38*100-(int(row.Pg_x+1)/int(row.Pg_y+1))))
    else:
        if round(int(row.Pg_y)/38*100+(int(row.Pg_y)/int(row.Pg_x+1))) > 100:
            prob.append(99.5)
        else:
            prob.append(round(int(row.Pg_y)/38*100+(int(row.Pg_y)/int(row.Pg_x+1))))

df['Probabile_titolarità_futura'] = prob # (occhio se il giocatore ha cambiato squadra)


In [27]:
prob = []
for index, row in df.iterrows():
    if row.Nome in giveatry:
        prob.append('Y - ex '+calciatori_old_dict[row.Nome])
    else:
        prob.append('N')

df['Squadra_nuova'] = prob

In [28]:
total = []
for index, row in df.iterrows():
    if row.mediaGiocatori > 0:
        total.append(row.mediaGiocatori * row.media * row['Mf_y'])
    else:
        total.append(0)

df['Convenienza_inizio_campionato'] = total # (considera solo le due annate precedenti concluse)

In [29]:
total = []
for index, row in df.iterrows():
    if row.mediaGiocatori > 0:
        total.append(row.mediaGiocatori_today * row.media_today * row['Mf'])
    else:
        total.append(0)

df['Convenienza_today'] = total

In [30]:
if giocateMax < 2:
    result = df.sort_values(by='Convenienza_inizio_campionato', ascending=False)
else:
    result = df.sort_values(by='Convenienza_today', ascending=False)

In [31]:
result = result.drop(['mediaGiocatori', 'media', 'mediaGiocatori_today', 'media_today'], axis=1)

output

In [32]:
df_P = result[result['R'] == "P"]
df_P.to_excel('data/output/dataset_P.xlsx')

In [33]:
df_D = result[result['R'] == "D"]
df_D.to_excel('data/output/dataset_D.xlsx')

In [34]:
df_C = result[result['R'] == "C"]
df_C.to_excel('data/output/dataset_C.xlsx')

In [35]:
df_A = result[result['R'] == "A"]
df_A.to_excel('data/output/dataset_A.xlsx')